# **ETL - COLOMBIA**

In [1]:
# [Configuración] Importación de librerías
import pandas as pd
import numpy as np
import unicodedata
import re
import os

In [2]:
# [Configuración] Lectura de los archivos CSV
ruta = "../DATA/raw_colombia" 
archivos = [f for f in os.listdir(ruta) if f.endswith(".csv")]
dataframes = {}
for archivo in archivos:
    path = os.path.join(ruta, archivo)
    df = pd.read_csv(path, encoding='Latin_1', low_memory=False)
    df.columns = df.columns.str.lower()
    dataframes[archivo] = df

print(f"✅ {len(dataframes)} archivos cargados desde {ruta}")

✅ 22 archivos cargados desde ../DATA/raw_colombia


In [3]:
# [Función] Limpiar nombres de columnas
def limpiar_texto(texto):
    if not isinstance(texto, str):
        return texto
    texto = texto.replace("ã±", "ñ").replace("Ã±", "ñ").replace("ï¿½", "ñ")
    texto = unicodedata.normalize("NFKC", texto)
    texto = re.sub(r'\s+', '_', texto.strip().lower())
    return texto

# Limpiar nombres de columnas de todos los DataFrames
for nombre, df in dataframes.items():
    df.columns = [limpiar_texto(col) for col in df.columns]
    dataframes[nombre] = df


In [4]:
# Descartar variables poco informativas
delete = [
    'ajuste', 'bar_ver', 'cbmte', 'cen_pobla', 'cer_def', 'cod_ase', 'cod_dpto_n', 'cod_dpto_o',
    'cod_dpto_r', 'cod_eve.1', 'cod_mun_n', 'cod_mun_o', 'cod_mun_r', 'cod_pais_o', 'cod_pais_r',
    'cod_pre', 'cod_sub', 'confirmados', 'consecutive', 'consecutive_12',
    'consecutive_origen', 'departamento_notificacion', 'departamento_residencia',
    'estado_final_de_caso', 'estrato', 'evento', 'fec_aju', 'fec_arc_xl', 'fec_def',
    'fm_fuerza', 'fm_grado', 'fm_unidad', 'fuente', 'gp_carcela', 'gp_desmovi',
    'gp_desplaz', 'gp_discapa', 'gp_gestan', 'gp_indigen', 'gp_mad_com', 'gp_migrant',
    'gp_otros', 'gp_pobicfb', 'gp_psiquia', 'gp_vic_vio', 'gru_pob', 'localidad',
    'municipio', 'municipio_notificacion', 'municipio_residencia', 'nacionalidad',
    'nom_est_f_caso', 'nom_grupo', 'nom_upgd', 'nombre_evento', 'nombre_nacionalidad',
    'nombre_upgd', 'ocupacion', 'pais_ocurrencia', 'pais_residencia', 'partición',
    'sem_ges', 'va_sispro', 'vereda', 'version', 'est_f_caso'
]
for nombre, data in dataframes.items():
    cols_a_borrar = [col for col in delete if col in data.columns]
    if cols_a_borrar:
        dataframes[nombre] = data.drop(columns=cols_a_borrar)
        print(f"🗑️ En '{nombre}' se eliminaron: {cols_a_borrar}")
    else:
        print(f"✅ En '{nombre}' no había columnas para eliminar.")

🗑️ En 'Datos_2013_210.csv' se eliminaron: ['ajuste', 'cbmte', 'cer_def', 'cod_ase', 'cod_dpto_n', 'cod_dpto_o', 'cod_dpto_r', 'cod_mun_n', 'cod_mun_o', 'cod_mun_r', 'cod_pais_o', 'cod_pais_r', 'cod_pre', 'cod_sub', 'confirmados', 'consecutive', 'consecutive_origen', 'departamento_notificacion', 'departamento_residencia', 'estado_final_de_caso', 'estrato', 'fec_aju', 'fec_arc_xl', 'fec_def', 'fm_fuerza', 'fm_grado', 'fm_unidad', 'fuente', 'gp_carcela', 'gp_desmovi', 'gp_desplaz', 'gp_discapa', 'gp_gestan', 'gp_indigen', 'gp_mad_com', 'gp_migrant', 'gp_otros', 'gp_pobicfb', 'gp_psiquia', 'gp_vic_vio', 'gru_pob', 'municipio', 'municipio_notificacion', 'municipio_residencia', 'nacionalidad', 'nom_est_f_caso', 'nom_grupo', 'nom_upgd', 'nombre_evento', 'nombre_nacionalidad', 'ocupacion', 'pais_ocurrencia', 'pais_residencia', 'sem_ges', 'va_sispro']
🗑️ En 'Datos_2013_220.csv' se eliminaron: ['ajuste', 'cbmte', 'cen_pobla', 'cer_def', 'cod_ase', 'cod_dpto_n', 'cod_dpto_o', 'cod_dpto_r', 'cod_m

## Unificación

In [5]:
# Concatenar archivos en un solo DataFrame
data = pd.concat(dataframes.values(), ignore_index=True)
data = data[sorted(data.columns)]
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 948142 entries, 0 to 948141
Data columns (total 18 columns):
 #   Column                 Non-Null Count   Dtype 
---  ------                 --------------   ----- 
 0   area                   948142 non-null  int64 
 1   año                    948142 non-null  int64 
 2   cod_eve                948142 non-null  int64 
 3   departamento           948142 non-null  object
 4   desenlace              948142 non-null  int64 
 5   edad                   948142 non-null  int64 
 6   fecha_consulta         948049 non-null  object
 7   fecha_hospitalizacion  400963 non-null  object
 8   fecha_inicio_sintomas  948089 non-null  object
 9   fecha_nacimiento       918020 non-null  object
 10  fecha_notificacion     948142 non-null  object
 11  hospitalizacion        948142 non-null  int64 
 12  pertenencia_etnica     948142 non-null  int64 
 13  regimen_salud          948142 non-null  object
 14  semana                 948142 non-null  int64 
 15  

In [6]:
# Eliminar registros duplicados exactos
count_duplicados = data.duplicated().sum()
print(f"Número de filas duplicadas exactas en columnas de interés: {count_duplicados}")
data = data.drop_duplicates(keep="first").copy()

Número de filas duplicadas exactas en columnas de interés: 351


__________

## Agregación

In [7]:
# [Variable] Evento
    # evento = Clasifico if code_eve = 210 
    # evento = Grave if code_eve = 220

data['evento'] = np.where(data['cod_eve'] == 210, 'Clasico', 
                       np.where(data['cod_eve'] == 220, 'Grave', 'Otros'))
data['evento'].unique()

array(['Clasico', 'Grave'], dtype=object)

In [8]:
# [Variable] Edad en años
data["edad_años"] = np.where(data["uni_med"] == 1, data["edad"],   # ya está en años
                    np.where(data["uni_med"] == 2, data["edad"] / 12,  # meses → años
                    np.where(data["uni_med"] == 3, data["edad"] / 365, # días → años
                    np.where(data["uni_med"] == 4, data["edad"] / (365*24), # horas
                    np.where(data["uni_med"] == 5, data["edad"] / (365*24*60), # minutos
                    np.where(data["uni_med"] == 0, np.nan, 
                     np.nan))))))  # por si hay otros valores
data["edad_años"] = data["edad_años"].round().fillna(-1).astype(int)
data['edad_años'].describe()

count    947791.000000
mean         22.887485
std          19.105988
min          -1.000000
25%           9.000000
50%          17.000000
75%          33.000000
max         131.000000
Name: edad_años, dtype: float64

In [9]:
# [Variable] Ciclo vital a partir de edad en años
def ciclo_vital(edad):
    if edad < 0:
        return "Desconocido"
    elif edad < 1:
        return "Menor de 1 año"
    elif edad < 5:
        return "Primera infancia"
    elif edad < 10:
        return "Infancia"
    elif edad < 20:
        return "Adolescencia"
    elif edad < 60:
        return "Adulto"
    elif edad < 85:
        return "Adulto mayor"
    else:
        return "Ancianidad"
    
data["ciclo_vital"] = data["edad_años"].apply(ciclo_vital)
data["ciclo_vital"].unique()

array(['Adulto mayor', 'Adulto', 'Adolescencia', 'Ancianidad', 'Infancia',
       'Menor de 1 año', 'Primera infancia', 'Desconocido'], dtype=object)

In [10]:
# [Mapeo] Variables categóricas
mapeo = {
    'area' : {1: 'Cabecera municipal' , 2:'Centro Poblado', 3:'Rural disperso'}, 
    'hospitalizacion': {1: 'Si', 2: 'No'},
    'desenlace': {1: 'Vivo', 2: 'Fallecido'},
    'pertenencia_etnica': {1: 'Indigena', 2: 'Rom', 3: 'Raizal', 4: 'Palenquero', 5: 'Negro', 6:'Otro'},
    'regimen_salud': {'p': 'Excepción', 'e': 'Especial', 'c': 'Contributivo', 's': 'Subsidiado', 'n': 'No asegurado', 'i': 'Indeterminado'},
    'sexo': {'m': 'Masculino', 'f': 'Femenino'},
    'tipo_caso': {2: 'Probable', 3: 'Conf. por laboratorio', 5: 'Conf. por nexo'}
}
data = data.replace(mapeo)

In [11]:
# [Mapeo] Grupos étnicos
mapeo_etnico = {
    'Indigena': 'Indígena',
    'Negro': 'Afrocolombiano',
    'Raizal': 'Afrocolombiano',
    'Palenquero': 'Afrocolombiano',
    'Rom': 'Rom',
    'Otro': 'Otro'
}
data['grupo_etnico'] = data['pertenencia_etnica'].map(mapeo_etnico)

In [12]:
# [Variable] Departamento

data['departamento'] = data["departamento"].str.upper().str.strip()
data['departamento'] = data['departamento'].str.replace('_', ' ', regex=False)

# Normalizar EXTERIOR y PROCEDENCIA DESCONOCIDA
data['departamento'] = data['departamento'].replace({
    'EXTERIOR': 'NO IDENTIFICADO',
    'PROCEDENCIA DESCONOCIDA': 'NO IDENTIFICADO',
})

correcciones_departamento = {
    'ATLANTICO': 'ATLÁNTICO',
    'BOGOTA': 'BOGOTÁ, D.C.',
    'BOLIVAR': 'BOLÍVAR',
    'BOYACA': 'BOYACÁ',
    'CAQUETA': 'CAQUETÁ',
    'CORDOBA': 'CÓRDOBA',
    'CHOCO': 'CHOCÓ',
    'GUAJIRA': 'LA GUAJIRA',
    'GUAINIA': 'GUAINÍA',
    'VAUPES': 'VAUPÉS',
    'NORTE SANTANDER': 'NORTE DE SANTANDER',
    'VALLE': 'VALLE DEL CAUCA',
    'SAN ANDRES': 'ARCHIPIÉLAGO DE SAN ANDRÉS, PROVIDENCIA Y SANTA CATALINA',
    'NARIÏ¿½O': 'NARIÑO', 'NARIÃ\x91O': 'NARIÑO',
}
data['departamento'] = data['departamento'].replace(correcciones_departamento)
data['departamento'].unique()

array(['ANTIOQUIA', 'ATLÁNTICO', 'SANTANDER', 'QUINDIO', 'ARAUCA',
       'CESAR', 'NORTE DE SANTANDER', 'CUNDINAMARCA', 'TOLIMA',
       'VALLE DEL CAUCA', 'META', 'CASANARE', 'HUILA', 'LA GUAJIRA',
       'GUAVIARE', 'CHOCÓ', 'MAGDALENA', 'CAUCA', 'SUCRE', 'BOYACÁ',
       'BOGOTÁ, D.C.', 'BOLÍVAR', 'CAQUETÁ', 'PUTUMAYO', 'CALDAS',
       'CÓRDOBA', 'AMAZONAS', 'NARIÑO', 'RISARALDA', 'VICHADA',
       'NO IDENTIFICADO',
       'ARCHIPIÉLAGO DE SAN ANDRÉS, PROVIDENCIA Y SANTA CATALINA',
       'GUAINÍA', 'VAUPÉS'], dtype=object)

In [13]:
# [Variable] Periodo epidemiológico (4 semanas)
def asignar_periodo(semana):
    """
    Calcula el número de periodo según la semana epidemiológica.
    Cada periodo abarca 4 semanas. 
    La semana 53 se incluye en el último periodo.
    """
    if pd.isna(semana):
        return np.nan
    semana = int(semana)
    periodo = int(np.ceil(semana / 4))
    if semana == 53:  # Semana extra del año
        periodo = int(np.ceil(52 / 4))  # igual al último periodo
    return periodo

# Aplicar al dataframe
data["periodo"] = data["semana"].apply(asignar_periodo)

In [14]:
# Descartar variables originales ya transformadas
data = data.drop(columns=['edad', 'uni_med', 'pertenencia_etnica', 'cod_eve'])

In [15]:
# [Data] Estructuración por temática

    # Variables demográficas
demograficas = [
    "sexo",                    # M/F
    "edad_años",               # Edad numérica
    "ciclo_vital",             # Etapa de vida
    "grupo_etnico",            # Grupo étnico declarado
    "pertenencia_etnica",      # Clasificación étnica
    "regimen_salud"            # Régimen (contributivo, subsidiado, etc.)
]

    # Variables del evento epidemiológico
evento = [
    "evento",                  # Clásico, grave
    "tipo_caso",               # Confirmado, probable, descartado
    "hospitalizacion",         # Sí/No
    "desenlace"                # Recuperado, fallecido, etc.
]

    # Variables de temporalidad
temporalidad = [
    "año",                     # Año del evento
    "semana",                  # Semana epidemiológica
    "periodo",                 # Periodo (cada 4 semanas)
    "fecha_inicio_sintomas",   # Inicio de síntomas
    "fecha_consulta",          # Fecha de consulta
    "fecha_hospitalizacion",   # Fecha de hospitalización
    "fecha_notificacion",      # Fecha de notificación
    "fecha_nacimiento"         # Fecha de nacimiento
]

    # Variables espaciales
espacial = [
    "departamento",            # Departamento de ocurrencia
    "area"                     # Urbana o rural
]

orden_columnas = demograficas + evento + temporalidad + espacial
cols_existentes = [col for col in orden_columnas if col in data.columns]
data = data[cols_existentes + [col for col in data.columns if col not in cols_existentes]]

In [16]:
# DataFrame final
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 947791 entries, 0 to 948141
Data columns (total 19 columns):
 #   Column                 Non-Null Count   Dtype 
---  ------                 --------------   ----- 
 0   sexo                   947791 non-null  object
 1   edad_años              947791 non-null  int64 
 2   ciclo_vital            947791 non-null  object
 3   grupo_etnico           947791 non-null  object
 4   regimen_salud          947791 non-null  object
 5   evento                 947791 non-null  object
 6   tipo_caso              947791 non-null  object
 7   hospitalizacion        947791 non-null  object
 8   desenlace              947791 non-null  object
 9   año                    947791 non-null  int64 
 10  semana                 947791 non-null  int64 
 11  periodo                947791 non-null  int64 
 12  fecha_inicio_sintomas  947738 non-null  object
 13  fecha_consulta         947698 non-null  object
 14  fecha_hospitalizacion  400805 non-null  object
 15  fecha

In [17]:
data.head()

,sexo,edad_años,ciclo_vital,grupo_etnico,regimen_salud,evento,tipo_caso,hospitalizacion,desenlace,año,semana,periodo,fecha_inicio_sintomas,fecha_consulta,fecha_hospitalizacion,fecha_notificacion,fecha_nacimiento,departamento,area
0,M,70,Adulto mayor,Otro,S,Clasico,Probable,Si,Vivo,2013,52,13,2013-04-05,2013-04-05,2013-04-06,2013-12-28,1942-11-17,ANTIOQUIA,Cabecera municipal
1,M,62,Adulto mayor,Otro,S,Clasico,Probable,No,Vivo,2013,46,12,2013-02-07,2013-02-10,NaN,2013-11-18,1950-02-12,ANTIOQUIA,Cabecera municipal
2,M,78,Adulto mayor,Otro,S,Clasico,Probable,No,Vivo,2013,9,3,2013-12-22,2013-12-25,NaN,2013-03-19,1935-04-12,ATLÁNTICO,Cabecera municipal
3,M,56,Adulto,Otro,S,Clasico,Conf. por laboratorio,Si,Vivo,2013,47,12,2013-07-31,2013-08-07,2013-08-07,2013-11-25,1957-03-29,ANTIOQUIA,Rural disperso
4,M,51,Adulto,Otro,C,Clasico,Probable,No,Vivo,2013,37,10,2013-05-07,2013-05-11,NaN,2013-09-12,1961-10-29,ANTIOQUIA,Cabecera municipal


In [18]:
data.to_csv(r"C:\Users\Hp\DENGUE\DATA\Data_Colombia.csv", index=False, encoding='utf-8')